In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler

# Define column names for dataset
col_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

# Load dataset
df = pd.read_csv(r"C:\Users\usuario\Downloads\census+income\adult.data", header=None, names=col_names)

# Strip whitespace from object type columns
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Initial data exploration
print(df.head())
print(df.isna().sum())  # Check for missing values
print(df.info())
print(df.shape)
print(df.describe())
print(df['income'].value_counts())

# Selecting features for the model
feature_cols = ['age', 'capital-gain', 'capital-loss', 'sex', 'race', 'hours-per-week', 'education']
x = pd.get_dummies(df[feature_cols], drop_first=True)  # Convert categorical variables into dummy/indicator variables

# Visualizing correlation matrix with heatmap
sns.set(font_scale=.65)
plt.figure(figsize=(15, 8))
sns.heatmap(x.corr(), annot=True, fmt=".2f", cmap='coolwarm', cbar=True, square=True)
plt.show()

# Feature scaling
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Create binary outcome variable
y = df['income'].apply(lambda x: 0 if x == '<=50K' else 1)

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, random_state=1, test_size=0.2)

# Initialize and train logistic regression model
lg = LogisticRegression(C=0.05, penalty='l1', solver='liblinear')
lg.fit(x_train, y_train)

# Predict on testing set
y_pred = lg.predict(x_test)

# Model evaluation
print('Model Parameters, Intercept:', lg.intercept_)
print('Model Parameters, Coefficients:', lg.coef_)
print('Confusion Matrix:', confusion_matrix(y_test, y_pred))
print('Accuracy:', accuracy_score(y_test, y_pred))

# Displaying model coefficients
coef_df = pd.DataFrame({'Variable': x.columns, 'Coefficient': lg.coef_[0]})
coef_df = coef_df[coef_df['Coefficient'] != 0].sort_values(by='Coefficient', ascending=False)
print(coef_df.head())

# Visualizing the coefficients as a bar plot
sns.set_style('whitegrid')
sns.barplot(x='Coefficient', y='Variable', data=coef_df)
sns.despine()
plt.tight_layout()
plt.show()

# ROC Curve and AUC
y_pred_prob = lg.predict_proba(x_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_prob)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()
print(f"ROC AUC: {roc_auc:.2f}")
